In [32]:
import random
from random import randint
import subprocess
import os
import signal
from subprocess import call
import time
import string
import redis

In [36]:
def getRandomString(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

def getRandomPortNum():
    return randint(30000,40000)

def getUniqueRandomPortNum():
    p = randint(30000,40000)
    while True:
        unique = True
        for s in server_list:
            p2 = int(s.split(':')[1])
            if p2 == p:
                unique = False
                break
        if unique: return p
        p = randint(30000,40000)
    


def startProc(args):
    return subprocess.Popen(args)

# p should be a process type
def killProc(proc):
    try:
        os.killpg(proc.pid+1, signal.SIGTERM)
#         call(["kill", "-9",str(proc.pid+1)]) #kill redis server pid+1 works for some reason
#         call(["kill", "-9", str(proc.pid)])
        return True
    except:
        print "Failed to kill proc pid:", proc.pid, " maybe doesn't exist"

def stopServer(addr):
    p = addr.split(':')[1]
    s = "redis-cli -p " + str(p) + " shutdown nosave"
    call(s.split())
    stopMonitoring(addr)
    

def killAllServers():
    for s in server_list:
        print "Killing server:",s," pid:", server_list[s].pid
        #killProc(server_list[s])
        stopServer(s)

def startServer(ip = "127.0.0.1", port=getRandomPortNum()):
    s = ("redis-server --port " + str(port) + " --appendonly yes --appendfilename appendonly-" + str(port) + ".aof"
    " --dbfilename dump-" + str(port) + ".rdb --logfile " + str(port) + ".log --daemonize yes --loglevel verbose")
#     print s
#     print s.split()
    p = startProc(s.split())
    sp = ip + ":" + str(port)
    print "Started server:",sp, " proc", p.pid
    server_list[sp] = p
    startMonitoring(sp)

def startServers(numServers):
    for i in range(numServers):
        startServer(port=getUniqueRandomPortNum())

        
def startMonitoring(addr):
    REDIS_SENTINEL.sentinel_monitor("masterP_" + addr, addr.split(':')[0], addr.split(':')[1], 2)
def stopMonitoring(addr):
    REDIS_SENTINEL.sentinel_remove("masterP_" + addr)

def cleanUp():
    call(["rm", "-rf", "*.log"])
    call(["rm", "-rf", "appendonly*.aof"])
    call(["rm", "-rf", "dump*.rdb"])
    call(["rm", "-rf", "nodes*.conf"])
    call(["rm", "-rf", "*.conf"])

In [ ]:
server_list = {}
REDIS_SENTINEL = redis.StrictRedis(host='localhost', port=40001)
cleanUp()
startServers(5)

In [ ]:
cleanUp()

In [ ]:
killAllServers()

In [ ]:

    
def getRandomString():
    
    
# def startSentinel():

    
def killSentinel():
    

#start redis + DT server + notify sentinel
#update server_list
    

def startServers(numServers):

    

#kill the redis server at addr
#update server_list
def killServer(addr):
    

#get and write the status to some log
def getStatus():
    
#compute num migrations from two succesie status calls and write to file
def getNumMigrations():